## Homework 9

In this homework, we'll deploy the dino or dragon model we trained in the previous homework.

Download the model from [here](https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5)

### Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the converted model?

- 21 Mb

- 43 Mb

- 80 Mb

- 164 Mb